<a href="https://colab.research.google.com/github/Shivayogi-A/Pyspark_programming/blob/master/Handling_null_values_in_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# create a spark session

from pyspark.sql import SparkSession
spark =SparkSession.builder\
       .appName("isNull & isNotNull")\
       .getOrCreate()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [920 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,552 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:14 h

# **isNull() and isNotNull()**

isNull() function is used to check if the current expression is NULL/None or column contains a NULL/None value, if it contains it returns a boolean value True. Similarly, isNotNull() function is used to check if the current expression is NOT NULL or column contains a NOT NULL value. if it contains any value it returns True.



In [4]:
# create Data Frame

data = [
    ("James",None,"M"),
    ("Anna","NY","F"),
    ("Julia",None,None)
  ]

columns = ["name","state","gender"]

df = spark.createDataFrame(data, columns)
df.show()

+-----+-----+------+
| name|state|gender|
+-----+-----+------+
|James| null|     M|
| Anna|   NY|     F|
|Julia| null|  null|
+-----+-----+------+




**isNull():** The function returns all the rows where cretain columns on which we apply this function contains null values.


Syntax of isNull()

* *Column.isNull()*


* *pyspark.sql.functions.isnull(col)*


**PySpark Column.isNull() Usage with Examples**\
To select rows that have a null value on a selected column use filter() with isNULL()

In [5]:
df.filter(df.state.isNull()).show()

+-----+-----+------+
| name|state|gender|
+-----+-----+------+
|James| null|     M|
|Julia| null|  null|
+-----+-----+------+



This can also be written in the below format and also we create a new dtaframe with the filtered records.

In [6]:
from pyspark.sql.functions import col
df2 = df.filter(col("State").isNull()).show()

+-----+-----+------+
| name|state|gender|
+-----+-----+------+
|James| null|     M|
|Julia| null|  null|
+-----+-----+------+



**Using Multiple Conditions**\
To filter rows with NULL values on multiple columns, use either AND or & operator.

Below we filter the records where both **State** and **gender** are null.

In [7]:
df3 = df.filter(col("state").isNull() & col("gender").isNull())
df3.show()

+-----+-----+------+
| name|state|gender|
+-----+-----+------+
|Julia| null|  null|
+-----+-----+------+



# PySpark isNotNull()

PySpark isNotNull() returns all the rows where there is no null values.


Syntax of isNotNull()
* *Column.isNotNull()*

The below example uses PySpark isNotNull() function from Column class to check if a column has a NOT NULL value.



In [8]:
df.filter(df.state.isNotNull()).show()

+----+-----+------+
|name|state|gender|
+----+-----+------+
|Anna|   NY|     F|
+----+-----+------+



In [9]:
df4 = df.filter(col("state").isNotNull())
df4.show()

+----+-----+------+
|name|state|gender|
+----+-----+------+
|Anna|   NY|     F|
+----+-----+------+



# Dropping rows with Null values - Dropna()

DataFrame/Dataset has a variable **na** which is an instance of class DataFrameNaFunctions hence, you should be using **na** variable on DataFrame to use drop().

DataFrameNaFunctions class also have method **fill()** to replace NULL values with empty string on PySpark DataFrame

In [13]:
#Example: create a dataframe by reading a file small_zipcode.csv

zp_df = spark.read\
            .format("csv")\
            .option("Header","True")\
            .option("inferschema","True")\
            .load("/content/drive/MyDrive/Colab_Notebooks/Datasets/small_zipcode.csv")
zp_df.show()


+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               null|   PR|     30100|
|  2|    704|    null|PASEO COSTA DEL SUR|   PR|      null|
|  3|    709|    null|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               null|   TX|      null|
+---+-------+--------+-------------------+-----+----------+



This yields the above output. If you notice, column name, city, and population have null values.

As you see above DataFrame most of the rows have NULL values except record with id=4. Now, let’s see how to drop or remove rows with null values on DataFrame.

By default drop() without arguments remove all rows that have null values on any column of DataFrame.

In [14]:
zp_df.dropna().show()

+---+-------+------+-----------------+-----+----------+
| id|zipcode|  type|             city|state|population|
+---+-------+------+-----------------+-----+----------+
|  4|  76166|UNIQUE|CINGULAR WIRELESS|   TX|     84000|
+---+-------+------+-----------------+-----+----------+



This removes all rows with null values and returns the clean DataFrame with id=4 where it doesn’t have any NULL values.

Alternatively you can also get same result with dropna:("any").

In [16]:
zp_df.dropna("any").show()

+---+-------+------+-----------------+-----+----------+
| id|zipcode|  type|             city|state|population|
+---+-------+------+-----------------+-----+----------+
|  4|  76166|UNIQUE|CINGULAR WIRELESS|   TX|     84000|
+---+-------+------+-----------------+-----+----------+





Below example drops all rows that has NULL values on all columns. Our DataFrame doesn’t have null values on all rows hence below examples returns all rows.

In [17]:
zp_df.dropna("all").show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               null|   PR|     30100|
|  2|    704|    null|PASEO COSTA DEL SUR|   PR|      null|
|  3|    709|    null|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               null|   TX|      null|
+---+-------+--------+-------------------+-----+----------+



**Drop Rows with NULL Values on Selected Columns**\
In order to remove Rows with NULL values on selected columns of PySpark DataFrame, use dropna(columns:Seq[String]) or dropna(columns:Array[String]). To these functions pass the names of the columns you wanted to check for NULL values to delete rows.

In [18]:
zp_df.dropna(subset = ['type']).show()

+---+-------+--------+-----------------+-----+----------+
| id|zipcode|    type|             city|state|population|
+---+-------+--------+-----------------+-----+----------+
|  1|    704|STANDARD|             null|   PR|     30100|
|  4|  76166|  UNIQUE|CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|             null|   TX|      null|
+---+-------+--------+-----------------+-----+----------+



In the above example it dropped only those rows where **type** column is null.

same way we can apply condition for multiple columns by mentioning the column names in subset as comma seperated values.

In [20]:
zp_df.dropna(subset = ['type','population',]).show()

+---+-------+--------+-----------------+-----+----------+
| id|zipcode|    type|             city|state|population|
+---+-------+--------+-----------------+-----+----------+
|  1|    704|STANDARD|             null|   PR|     30100|
|  4|  76166|  UNIQUE|CINGULAR WIRELESS|   TX|     84000|
+---+-------+--------+-----------------+-----+----------+



# Replacing null values with a delfault value in Pyspark -fillna()

In PySpark,fillna() from DataFrame class or fill() from DataFrameNaFunctions is used to replace NULL/None values on all or selected multiple columns with either zero(0), empty string, space, or any constant literal values.

While working on data cleaning we may need to replace the null values as one of the preliminary step. We can use the below method to do the same.



**PySpark fillna() & fill() Syntax**\
PySpark provides DataFrame.fillna() and DataFrameNaFunctions.fill() to replace NULL/None values. These two are aliases of each other and returns the same results.

syntax:\
*fillna(value, subset=None)*\
*fill(value, subset=None)*


**value** – Value should be the data type of int, long, float, string, or dict. Value specified here will be replaced with NULL/None values.

**subset** – This is optional, when used it should be the subset of the column names where you wanted to replace NULL/None values.

In [24]:
zp_df.fillna(value = 0).show()
zp_df.printSchema()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               null|   PR|     30100|
|  2|    704|    null|PASEO COSTA DEL SUR|   PR|         0|
|  3|    709|    null|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               null|   TX|         0|
+---+-------+--------+-------------------+-----+----------+

root
 |-- id: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- population: integer (nullable = true)



Note in the above example the value null was replaced with only in popluation column as the data type of it is an integer.

**PySpark Replace Null/None Value with Empty String**



In [25]:
zp_df.fillna(" ").show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|                   |   PR|     30100|
|  2|    704|        |PASEO COSTA DEL SUR|   PR|      null|
|  3|    709|        |       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|                   |   TX|      null|
+---+-------+--------+-------------------+-----+----------+



Now, let’s replace NULLs on specific columns, below example replace a column type with an empty string and a column city with the value “unknown”.

In [27]:
zp_df.fillna("unknown",['type'])\
      .fillna(" ",subset = ['city'])\
      .show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|                   |   PR|     30100|
|  2|    704| unknown|PASEO COSTA DEL SUR|   PR|      null|
|  3|    709| unknown|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|                   |   TX|      null|
+---+-------+--------+-------------------+-----+----------+



Alternatively, write the above statement by using the terms in a form of dictionary as below

In [29]:
zp_df.fillna({"city": "unknown", "type": ""}) \
    .show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|            unknown|   PR|     30100|
|  2|    704|        |PASEO COSTA DEL SUR|   PR|      null|
|  3|    709|        |       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|            unknown|   TX|      null|
+---+-------+--------+-------------------+-----+----------+

